### <font color = 'purple'> DATA 255: HOMEWORK 09 Transformer Based Text Translation

#### <font color = 'purple'> Step1. Run the demo and train a model on the original German-to-English training set.

In [1]:
pip uninstall -q -y torch torchtext

<font color = 'blue'> Downloaded torch and torchtext with versions compatible with each other. Torchtext last version is 0.18.0 is not compatible with latest torch version. So downgrading to torch 2.3.0 to work with torchtext

In [2]:
pip install -q torch==2.3.0 torchtext==0.18.0 torchdata spacy portalocker==2.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [3]:
# !rm -f /kaggle/working/*

In [4]:
import torch
import torchtext
print(torch.__version__)
print(torchtext.__version__)

2.3.0+cu121
0.18.0+cpu


<font color='blue'> Downloading English language pack using spacy

In [5]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 97.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


<font color='blue'> Downloading German language pack using spacy

In [6]:
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 53.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


<font color='blue'> Importing relevant libraries

In [7]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from timeit import default_timer as timer

/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [8]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

<font color='blue'> Setting up train and validation dataset using Multi30K dataset

In [9]:
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

token_transform = {}
vocab_transform = {}

<font color='blue'> Initializing tokenizers for English and German Languages and builing vocabulary for training

In [10]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

# special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3

special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']


for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Setting UNK_IDX as the default index.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln].set_default_index(UNK_IDX)

/usr/local/lib/python3.10/dist-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [11]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

<font color='blue'> Defining the transformer model

In [12]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [13]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

<font color='blue'> Initialize the Model, loss function and optimizer

In [14]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

transformer_de_en = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer_de_en.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer_de_en = transformer_de_en.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(transformer_de_en.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [15]:
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))

text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln],
                                               vocab_transform[ln],
                                               tensor_transform)

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

<font color='blue'> Function for the training and evaluation batch

In [16]:
def train_epoch(model, optimizer):
    model.train()
    losses = 0
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in train_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in val_dataloader:
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

<font color='blue'> Train the Model and evaluate the model's performance on validation batch

In [17]:
NUM_EPOCHS = 21

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer_de_en, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer_de_en)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

def translate(model: torch.nn.Module, src_sentence: str, SRC_LANG: str, TGT_LANG: str):
    model.eval()
    src = text_transform[SRC_LANG](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANG].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


Epoch: 1, Train loss: 5.344, Val loss: 4.107, Epoch time = 42.197s
Epoch: 2, Train loss: 3.760, Val loss: 3.309, Epoch time = 41.990s
Epoch: 3, Train loss: 3.157, Val loss: 2.886, Epoch time = 43.635s
Epoch: 4, Train loss: 2.767, Val loss: 2.639, Epoch time = 44.950s
Epoch: 5, Train loss: 2.477, Val loss: 2.439, Epoch time = 44.069s
Epoch: 6, Train loss: 2.247, Val loss: 2.305, Epoch time = 45.216s
Epoch: 7, Train loss: 2.055, Val loss: 2.208, Epoch time = 44.058s
Epoch: 8, Train loss: 1.893, Val loss: 2.114, Epoch time = 44.184s
Epoch: 9, Train loss: 1.754, Val loss: 2.053, Epoch time = 45.254s
Epoch: 10, Train loss: 1.628, Val loss: 2.007, Epoch time = 43.996s
Epoch: 11, Train loss: 1.519, Val loss: 1.961, Epoch time = 44.807s
Epoch: 12, Train loss: 1.419, Val loss: 1.955, Epoch time = 44.723s
Epoch: 13, Train loss: 1.330, Val loss: 1.969, Epoch time = 44.051s
Epoch: 14, Train loss: 1.245, Val loss: 1.973, Epoch time = 44.925s
Epoch: 15, Train loss: 1.173, Val loss: 1.931, Epoch time

<font color = 'blue'> From the model performance, we see that there is a consistent decrease in training loss across all epochs indicating that the model is learning the patterns in the training data effectively. Whereas validation loss improves significantly until epoch 16, where it starts to plateau and slightly increase or fluctuate which suggests potential overfitting. The best performance we see at around epoch 16 (Val loss: 1.895). We can consider early stopping at this point to avoid overfitting.

<font color = 'blue'> We will see a few examples on how the model is translating text from German to English

In [18]:
print(translate(transformer_de_en,
                "Eine Gruppe von Menschen steht vor einem Iglu .",
                SRC_LANGUAGE,
                TGT_LANGUAGE))

 A group of people standing in front of an igloo 


<font color = 'blue'> Calling the translate function with the parameters as model, text and the source language and target language

In [19]:
print(translate(transformer_de_en,
                "Ein Junge mit Kopfhörern sitzt auf den Schultern einer Frau.",
                SRC_LANGUAGE,
                TGT_LANGUAGE))

 A boy with headphones is sitting on a woman 's shoulders . 


<font color = 'blue'> Saving the model state

In [20]:
torch.save(transformer_de_en.state_dict(), "transformer_de_en.pth")

#### <font color = 'purple'>Step2. Train a New model of the same architecture on the opposite training set (English-to-German)

<font color = 'blue'>Training a new model on the same architecture for English-to-German Translation

In [21]:
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'de'

<font color = 'blue'> Resetting token and vocabulary transformations for new language direction

In [22]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [23]:
vocab_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # new language pair in Multi30k dataset
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

<font color = 'blue'> Initializing and configuring the Seq2SeqTransformer for English-to-German training

In [24]:
SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])

transformer_en_de = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM).to(DEVICE)

for p in transformer_en_de.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

optimizer = torch.optim.Adam(transformer_en_de.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


<font color = 'blue'> Redefining the text transformation for the new source-target languages

In [25]:
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln],
                                               vocab_transform[ln],
                                               tensor_transform)


<font color = 'blue'> Training the model with the English-to-German dataset

In [26]:
NUM_EPOCHS = 21
for epoch in range(1, NUM_EPOCHS + 1):
    start_time = timer()
    train_loss = train_epoch(transformer_en_de, optimizer)
    end_time = timer()
    val_loss = evaluate(transformer_en_de)
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "
           f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 5.881, Val loss: 4.637, Epoch time = 51.660s
Epoch: 2, Train loss: 4.125, Val loss: 3.778, Epoch time = 51.197s
Epoch: 3, Train loss: 3.460, Val loss: 3.318, Epoch time = 51.532s
Epoch: 4, Train loss: 3.028, Val loss: 3.025, Epoch time = 52.579s
Epoch: 5, Train loss: 2.707, Val loss: 2.815, Epoch time = 51.784s
Epoch: 6, Train loss: 2.451, Val loss: 2.667, Epoch time = 51.463s
Epoch: 7, Train loss: 2.243, Val loss: 2.527, Epoch time = 51.646s
Epoch: 8, Train loss: 2.063, Val loss: 2.414, Epoch time = 51.915s
Epoch: 9, Train loss: 1.910, Val loss: 2.326, Epoch time = 52.694s
Epoch: 10, Train loss: 1.778, Val loss: 2.267, Epoch time = 51.755s
Epoch: 11, Train loss: 1.660, Val loss: 2.219, Epoch time = 51.796s
Epoch: 12, Train loss: 1.553, Val loss: 2.173, Epoch time = 51.707s
Epoch: 13, Train loss: 1.453, Val loss: 2.154, Epoch time = 52.734s
Epoch: 14, Train loss: 1.363, Val loss: 2.151, Epoch time = 52.359s
Epoch: 15, Train loss: 1.288, Val loss: 2.118, Epoch time

<font color = 'blue'> The training process for the English-to-German Transformer model shows steady progress, with training loss and validation loss decreasing over the epochs which indicates effective learning. By epoch 16, the validation loss reaches its minimum value of 2.060, suggesting this is the point of optimal performance. Later, the validation loss begins to plateau and slightly increase, with minimal improvements in validation performance. This suggests same that the model is starting to overfit the training data after 16th epoch.

In [27]:
torch.save(transformer_en_de.state_dict(), "transformer_en_de.pth")

<font color = 'blue'> Calling the translate function with the parameters as model, text and the source language and target language

In [28]:
print(translate(transformer_en_de,
                "A group of people stands in front of an igloo.",
                SRC_LANGUAGE,
                TGT_LANGUAGE))

 Eine Gruppe von Menschen steht vor einem Iglu . 


#### <font color = 'purple'>Step3. Insert novel sentences into your English-to-German model. Take the output and feed it to the original German-to-English model. Observe and report qualitatively on the results**

<font color = 'blue'> Sample sentences to test the English-to-German and back-translated German-to-English pipeline

In [29]:
test_sentences = [
    "The cat is sleeping on the couch.",
    "A man is riding a bicycle through the park.",
    "The sky is clear, and the sun is shining brightly.",
    "She is reading a book in the library.",
    "A child is playing with a ball on the beach."
]

<font color = 'blue'> First, translate from English to German using the English-to-German model in step-2

In [30]:
translated_sentences = []
for sentence in test_sentences:
    german_translation = translate(transformer_en_de, sentence, 'en', 'de')
    translated_sentences.append(german_translation)
    print(f"English to German: '{sentence}' -> '{german_translation}'")

English to German: 'The cat is sleeping on the couch.' -> ' Die Katze schläft auf dem Sofa . '
English to German: 'A man is riding a bicycle through the park.' -> ' Ein Mann fährt mit einem Fahrrad durch den Park . '
English to German: 'The sky is clear, and the sun is shining brightly.' -> ' Der Himmel ist klaren , die Sonne gekleidet ist , und die Sonne scheint . '
English to German: 'She is reading a book in the library.' -> ' Sie liest ein Buch in der Bibliothek . '
English to German: 'A child is playing with a ball on the beach.' -> ' Ein Kind spielt am Strand mit einem Ball . '


<font color = 'blue'> Now, take the German translations and translate back to English using the German-to-English model in step-1

In [31]:
print("\n--- German to English (Back-translation) ---")
for original, german_sentence in zip(test_sentences, translated_sentences):
    back_translation = translate(transformer_de_en, german_sentence, 'de', 'en')
    print(f"Original: '{original}'")
    print(f"German Translation: '{german_sentence}'")
    print(f"Back to English: '{back_translation}'\n")


--- German to English (Back-translation) ---
Original: 'The cat is sleeping on the couch.'
German Translation: ' Die Katze schläft auf dem Sofa . '
Back to English: ' The cat is sleeping on the couch . '

Original: 'A man is riding a bicycle through the park.'
German Translation: ' Ein Mann fährt mit einem Fahrrad durch den Park . '
Back to English: ' A man is riding a bicycle through the park . '

Original: 'The sky is clear, and the sun is shining brightly.'
German Translation: ' Der Himmel ist klaren , die Sonne gekleidet ist , und die Sonne scheint . '
Back to English: ' There are the sun , the sun , dressed in the sun , and the sun . '

Original: 'She is reading a book in the library.'
German Translation: ' Sie liest ein Buch in der Bibliothek . '
Back to English: ' It is reading a book in the library . '

Original: 'A child is playing with a ball on the beach.'
German Translation: ' Ein Kind spielt am Strand mit einem Ball . '
Back to English: ' A child playing with a ball on a 

<font color = 'blue'> The results of the English-to-German and back-translation to English shows mixed performance. For some sentences, the model successfully captures the meaning and structure during the forward and backward translations whereas for others, inaccuracies or semantic shifts do occur.

<font color = 'blue'> <b> High-quality translations:

<font color = 'blue'> Sentences like "The cat is sleeping on the couch." and "A man is riding a bicycle through the park." are accurately translated to German and back to English indicating model is able to handle straightforward sentences with simple syntax.

<font color = 'blue'><b> Moderate-quality translations:

<font color = 'blue'>In "A child is playing with a ball on the beach.", the back-translated sentence slightly deviates from the original by dropping the article "A" before "child" in the English reconstruction. However, the overall meaning was preserved.

<font color = 'blue'><b> Low-quality translations:

<font color = 'blue'>
The sentence "The sky is clear, and the sun is shining brightly." undergo significant distortion during back-translation. Phrases like "This sun is dressed in the sky" do not capture coherence suggesting model is struggling to translate complex sentences. Similarly "She is reading a book in the library." was back-translated incorrectly as "It is a book reading a book in the library," indicating challenges with pronouns and verbs.

<font color = 'blue'>  The model handles simple declarative sentences well, maintaining translation consistency and demonstrating a solid grasp of basic structure and vocabulary but struggles with complex sentences, showing limitations in understanding nuanced relationships

<font color = 'blue'>  Further fine-tuning of the models on a dataset with diverse sentence structures could help address these challenges.

## <font color = 'blue'> Thank You!!